In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt


In [6]:
strava_data = pd.read_csv('strava_data.csv')

In [15]:
strava_data.reset_index().plot(x='index', y='elapsed_time')
plt.show()

TypeError: Empty 'DataFrame': no numeric data to plot